In [1]:
import grequests
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests



In [2]:
from selenium import webdriver
from selenium.webdriver.chrome import service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException as WDE
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException


import pandas as pd
import time
from selenium.webdriver.common.by import By


In [3]:
options = Options()
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--headless")
options.add_argument("--disable-dev-shm-usage")

In [4]:
songdata = pd.read_csv('/opt/ml/finalproject/kakaomusic/songs2.csv')
songdata.head(5)

,song_id,song_title,artist,song_genre,playlist_tag,playlist_title,lyric,id
0,579571,Bezerk (Feat. A$AP Ferg),Big Sean,랩/힙합,3105|40027091|5년동안|8281|HipHop|Hot100|Party|hi...,외국 힙합 띵곡 모음|듣기만해도 3대500할 것 같은 힙합!|턴업!! 춤추고 싶게 ...,"Yeah, yeah (Hit-Boy)| Woah yeah, yeah| Sean, w...",0
1,579698,Take U There (Feat. Kiesza) (Missy Elliott Rem...,Jack U,일렉트로니카,2019UMF|Club|EDM|Electronic|Festival|GZ|HipHop...,#어디가서 힙알못 소리는 듣지말자 (해외)|#11 foreign swag songs...,You show me there's something more to us|Than ...,1
2,580131,King Of My Heart,Taylor Swift,POP,EDM|Hook|Pop|RnB|electronica|감성|개강|고등학생|공연|국외영...,해외에서 들으면 느낌 쩌는 취향저격 EDM & PoP|여행 갈때 이 곡과 떠나요~!...,I'm perfectly fine I live on my own|I made up ...,2
3,581120,Not My Friend (Feat. PLUTO),땡우,인디음악,2016년도1분기|MC|OwenOvadoz|겨울|결산|공연|국내외|국내힙합|기분전환...,"대학교 축제 기간! 주점에서 플레이하기 좋은 알앤비, 힙합|힙합 틀어주세여 현기증난...",i came from LEVEL 0|대놓고 면전에다|욕하는 놈 어딨겠냐만|다 써져있...,3
4,581409,Pretend,Mali-Koa Hood,POP,Chill|Pop|감동|감미로운|감성|기분전환|기분좋은|나른한노래|달닻|독서할때|드...,힘든 청춘에게 바치는 랩소디|잔잔한 바람과 차분하게 듣기좋은 음악|[POP/KPOP...,unk,4


In [7]:
# songdata.iloc[30:50]

In [7]:
song = songdata[['id','artist','song_title']]
song.head(5)

,id,artist,song_title
0,0,Big Sean,Bezerk (Feat. A$AP Ferg)
1,1,Jack U,Take U There (Feat. Kiesza) (Missy Elliott Rem...
2,2,Taylor Swift,King Of My Heart
3,3,땡우,Not My Friend (Feat. PLUTO)
4,4,Mali-Koa Hood,Pretend


In [8]:
song_ll = song.values.tolist()

In [8]:
for row in song_ll:
    # print(row)
    query = row[1] + ' ' + row[2]
    print(query)
    break

Big Sean Bezerk (Feat. A$AP Ferg)


In [9]:
batch1 = song_ll[12501:]
# batch2 = song_ll[20000:]
print(batch1[0])
# print(len(batch2))


[12501, '신승훈', '처음 그 느낌처럼']


In [13]:
20000%5000
20000//5000

4

In [17]:
song_ll[781:784]

[[781, 'NO:EL', '長龍峻 (장용준)'],
 [782, '#안녕', '생각이 난다'],
 [783,
  '잔잔한 피아노',
  'Mozart : Piano Sonata No.11 In A Major K.331 - I. Tema (모차르트 : 피아노 소나타 11번 가장조 - 1악장)']]

In [11]:
url_df = pd.DataFrame(columns = {'id','url'})
urls = []
baseurl = 'https://www.youtube.com/results?search_query='
cnt = 0

s=Service('/opt/ml/chromedriver')
driver = webdriver.Chrome(options=options, service=s)
driver.implicitly_wait(5)



for idx, row in enumerate(tqdm(batch1)):
    # k = song_ll[782]
    songid = row[0]
    query = row[1] + ' ' + row[2]
    # songid = k[0]
    # query = k[1] + ' ' + k[2]
    url = baseurl + query

    if idx % 100 == 99:
        driver.quit()
        s=Service('/opt/ml/chromedriver')
        driver = webdriver.Chrome(options=options, service=s)
        driver.implicitly_wait(5)

    driver.get(url)
    dic = {}
    try:
        a = driver.find_element_by_xpath('//*[@id="video-title"]').get_attribute('href') # 최신영상 접속
        dic['id'] = songid
        dic['url'] = a
        url_df = url_df.append(dic, ignore_index=True)
    except NoSuchElementException as n:
        dic['id'] = songid
        dic['url'] = None
        url_df = url_df.append(dic, ignore_index=True)
    # print(dic)
    # break
    if(idx%2500 == 0):
        serialnum = idx//2500
        nnn = '12501~' + str(serialnum*2500) +'_music_url.csv'
        url_df.to_csv(nnn, index=False)

 
 
        

  0%|          | 0/27499 [00:00<?, ?it/s]/tmp/ipykernel_17243/251069523.py:29: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  a = driver.find_element_by_xpath('//*[@id="video-title"]').get_attribute('href') # 최신영상 접속
100%|██████████| 27499/27499 [17:42:43<00:00,  2.32s/it]   


In [12]:
nnn = '12501~' + str(27500) +'_music_url.csv'
url_df.to_csv(nnn, index=False)

In [13]:
song_ll[-1]

[39999, 'Cheat Codes', 'Balenciaga']

In [3]:
a = pd.read_csv('/opt/ml/finalproject/youtubecrawl/12500_music_url.csv')
b = pd.read_csv('/opt/ml/finalproject/youtubecrawl/12501~27500_music_url.csv')

In [4]:
full = pd.concat([a,b], ignore_index=True)
full

,url,id
0,https://www.youtube.com/watch?v=oak7VyEHvro,0.0
1,https://www.youtube.com/watch?v=r89yS7-1Q8A,1.0
2,https://www.youtube.com/watch?v=5U7bF68xcRg,2.0
3,https://www.youtube.com/watch?v=SS5WtM21RHQ,3.0
4,https://www.youtube.com/watch?v=SEZsww9F_yc,4.0
...,...,...
39995,https://www.youtube.com/watch?v=Jl0U7F27G3U,39995.0
39996,https://www.youtube.com/watch?v=yAjVntEUPyE,39996.0
39997,https://www.youtube.com/watch?v=JYko5hDmN7Q,39997.0
39998,https://www.youtube.com/watch?v=t1g5W5ID6f0,39998.0


In [5]:
urls = full[['url']]
urls

,url
0,https://www.youtube.com/watch?v=oak7VyEHvro
1,https://www.youtube.com/watch?v=r89yS7-1Q8A
2,https://www.youtube.com/watch?v=5U7bF68xcRg
3,https://www.youtube.com/watch?v=SS5WtM21RHQ
4,https://www.youtube.com/watch?v=SEZsww9F_yc
...,...
39995,https://www.youtube.com/watch?v=Jl0U7F27G3U
39996,https://www.youtube.com/watch?v=yAjVntEUPyE
39997,https://www.youtube.com/watch?v=JYko5hDmN7Q
39998,https://www.youtube.com/watch?v=t1g5W5ID6f0


In [16]:
urls.iloc[99]

url    NaN
Name: 99, dtype: object

In [6]:
url_ll = urls.values.tolist()

In [9]:
url_ll = np.array(url_ll)
url_ll

array([['https://www.youtube.com/watch?v=oak7VyEHvro'],
       ['https://www.youtube.com/watch?v=r89yS7-1Q8A'],
       ['https://www.youtube.com/watch?v=5U7bF68xcRg'],
       ...,
       ['https://www.youtube.com/watch?v=JYko5hDmN7Q'],
       ['https://www.youtube.com/watch?v=t1g5W5ID6f0'],
       ['https://www.youtube.com/watch?v=tLA83s4L2pI']], dtype='<U43')

In [12]:
url_ll = url_ll.flatten()

In [20]:
cnt = 0
for i in tqdm(url_ll):
    if i == 'nan':
        cnt += 1
cnt

100%|██████████| 40000/40000 [00:00<00:00, 596432.05it/s]


304

In [21]:
set_url = set(url_ll)
len(set_url)

36393

In [22]:
full.to_csv('full_music_urls.csv')

In [10]:
full = pd.read_csv('full_music_urls.csv')
full[['url']].value_counts()

url                                        
https://www.youtube.com/watch?v=2jMdRxeZEkQ    17
https://www.youtube.com/watch?v=Lq9mIkLa8Fc    17
https://www.youtube.com/watch?v=tCarlLNwekA    16
https://www.youtube.com/watch?v=9vMh9f41pqE    15
https://www.youtube.com/watch?v=97_VJve7UVc    13
                                               ..
https://www.youtube.com/watch?v=dvOd_ucf10g     1
https://www.youtube.com/watch?v=dvSZQ4oMHGM     1
https://www.youtube.com/watch?v=dvWIbzqft7w     1
https://www.youtube.com/watch?v=dvf--10EYXw     1
https://www.youtube.com/watch?v=Uu_Ab2z-RMc     1
Length: 36392, dtype: int64